# STEP 40 브로드캐스트 함수
## 40.1 breadcast_to 함수와 sum_to 함수(넘파이 버전)
* np.broadcast_to(x, shape)
  * x의 원소를 복제하여 shape 인수로 지정한 형상이 되도록 함

In [1]:
import numpy as np

x = np.array([1, 2, 3])
y = np.broadcast_to(x, (2, 3))
print('x : ', x)
print('y : \n', y)

x :  [1 2 3]
y : 
 [[1 2 3]
 [1 2 3]]


* 브로드캐스트가 수행된 후의 역전파는?
    * '원소 복사'가 일어난 경우 역전파는 *기울기의 합*
    * 같은 변수를 여러번 사용하는 것에 대한 역전파 처리가 되어 있음
        * ex> y = x + x
            * 역전파에서는 x에 기울기를 두 번 흘려보내 더해지게 됨(14단계)
    * ![](../../images/그림%2040-1.png)
    * 브로드캐스트의 역전파 목표
        * x의 형상과 같아지도록 기울기의 합을 구함
        * sum_to(x, shape) 함수
            * sum_to는 x의 원소의 합을 구해 shape 형상으로 만들어주는 함수

In [2]:
import numpy as np
from dezero.utils import sum_to

x = np.array([[1,2,3], [4,5,6]])
y = sum_to(x, (1,3))
print(y)

y = sum_to(x, (2,1))
print(y)

[[5 7 9]]
[[ 6]
 [15]]


* sum_to 함수의 역전파
    * ![](../../images/그림%2040-2.png)

## 40.2 broadcast_to 함수와 sum_to 함수(DeZero 버전)
* broadcast_to 함수, BroadcastTo 클래스 구현
```python
class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = np.broadcast_to(x, self.shape)
        return y

    def backward(self, gy):
        gx = num_to(gy, self.x_shape)
        return gx

def broadcast_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return BroadcastTo(shape)(x)
```
* sum_to 함수와 SumTo 클래스 구현
```python
from dezero import utils

class SumTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = utils.sum_to(x, self.shape)
        return y

    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx

def sum_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return SumTo(shape)(x)
```

## 40.3 브로드캐스트 대응
* 브로드캐스트
    * 형상이 다른 다차원 배열끼리의 연산이 가능

In [3]:
from dezero import Variable

x0 = Variable(np.array([1,2,3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)
y.backward()
print(x0.grad) # 실행시점에는 구현되어져 있음
print(x1.grad)

variable([11 12 13])
variable([1 1 1])
variable([3])


* 순전파는 ndarray 인스턴스를 사용해 구현했기 때문에 브로드캐스트가 일어남
* 현재의 DeZero에서 브로드캐스트의 역전파는 일어나지 않음
```python
class Add(Function):
    def forward(self, x0, x1):
        self.x0_shape, self.x1_shape = x0.shape, x1.shape
        y = x0 + x1
        return y

    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x0_shape != self.x1_shape:
            gx0 = dezero.functions.sum_to(gx0, self.x0_shape)
            gx1 = dezero.functions.sum_to(gx1, self.x1_shape)
        return gx0, gx1
```
* 사칙연산 클래스(Mul, Sub, Div)에 모두 같은 수정 반영